# BERT IMPLEMENTATION USING SIMPLE TRANSFORMER LIBRARY

1. The Simple Transformers library is built on top of the excellent Transformers library by Hugging Face.
2. It uses Pytorch at the backend.
3. Simple Transformers Library acts a wrappper over the Transformers Library by Hugging Face.


#Evaluation Data
Used with *eval_model()*

###The evaluation data format is identical to the train data format.

##**Binary classification**
text	                          labels

Aragorn was the heir of Isildur	1

Frodo was the heir of Isildur	  0

##**Multi-class classification**
text	                         labels

Aragorn was the heir of Isildur	1

Frodo was the heir of Isildur	  0

Pippin is stronger than Merry	  2

##**Regression**
text	                          labels

Aragorn was the heir of Isildur	 1.0

Frodo was the heir of Isildur	   0.0

Pippin is stronger than Merry	   0.3

##**Multi-label classification**
text	                          labels

Aragorn was the heir of Isildur	[0, 1]

Frodo was the heir of Isildur	  [0, 0]

Pippin is stronger than Merry	  [1, 1]


In [1]:
!pip install -q simpletransformers

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score

In [3]:
#data = pd.read_csv("/content/imdb_sentiment.csv")
df = pd.read_csv("/content/amazon_reviews_us_Jewelry_v1_00.tsv", sep='\t',on_bad_lines='skip')

In [4]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5.0,0.0,0.0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31
1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5.0,0.0,0.0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31
2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5.0,0.0,0.0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31
3,US,5350721,R2I150CX5IVY9Q,B0006SW2WU,569859289,Surgical Stainless Steel Domed 9mm Fishbone Ri...,Jewelry,5.0,0.0,0.0,N,Y,Five Stars,Love it. Fits great. Super comfortable and nea...,2015-08-31
4,US,24484424,R1RM9ICOOA9MQ3,B009YPDW70,332947422,"Sterling Silver Family Pendant Necklace, 18""",Jewelry,5.0,0.0,0.0,N,Y,... a Mother's Day gift for my Mom and she lov...,Got this as a Mother's Day gift for my Mom and...,2015-08-31


In [5]:
df["star_rating"].unique()

array([ 5.,  1.,  4.,  3.,  2., nan])

In [6]:
df["Sentiment"] = df["star_rating"].apply(lambda score: "positive" if score >=3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,Sentiment
0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5.0,0.0,0.0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31,1
1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5.0,0.0,0.0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31,1
2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5.0,0.0,0.0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31,1
3,US,5350721,R2I150CX5IVY9Q,B0006SW2WU,569859289,Surgical Stainless Steel Domed 9mm Fishbone Ri...,Jewelry,5.0,0.0,0.0,N,Y,Five Stars,Love it. Fits great. Super comfortable and nea...,2015-08-31,1
4,US,24484424,R1RM9ICOOA9MQ3,B009YPDW70,332947422,"Sterling Silver Family Pendant Necklace, 18""",Jewelry,5.0,0.0,0.0,N,Y,... a Mother's Day gift for my Mom and she lov...,Got this as a Mother's Day gift for my Mom and...,2015-08-31,1


In [7]:
df["Sentiment"].unique()

array([1, 0])

In [8]:
df["Sentiment"].value_counts()  #need to data balance, if not, not proper Accuracy.

Sentiment
1    859552
0    145229
Name: count, dtype: int64

In [9]:
#df['short_review'] =df['review_body'].str.decode("utf-8")
df['short_review'] =df['review_body']

In [10]:
df = df[["short_review", "Sentiment"]]
df.head()

,short_review,Sentiment
0,so beautiful even tho clearly not high end ......,1
1,"Great product.. I got this set for my mother, ...",1
2,Exactly as pictured and my daughter's friend l...,1
3,Love it. Fits great. Super comfortable and nea...,1
4,Got this as a Mother's Day gift for my Mom and...,1


In [11]:
#Independent and dependent variable
x = df["short_review"]
y = df["Sentiment"]

In [12]:
#x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.99)

In [13]:
x_test

482158                           It is everything I wanted.
821603                                           loved this
200291    Very pretty and sparkly.  . It has an antique ...
931243                               Nice piece of jewelry.
754822    I love these. They are very fashionable and co...
                                ...                        
260376    Very pretty shipped fast and fits perfect-true...
67664                                                  NICE
930686    I have returned this jewelry item.  It was lar...
779122    thin metal. I ordered a necklace and received ...
372085    The earring/necklace combo is nice and stylish...
Name: short_review, Length: 994734, dtype: object

In [14]:
!pip install wandb -qU

In [15]:
# Log in to your W&B account
import wandb

In [16]:
wandb.login()

wandb: Currently logged in as: samim1000 (samim1234). Use `wandb login --relogin` to force relogin


True

In [17]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [18]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.learning_rate = 1e-4

In [19]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [20]:
### Advanced Methodology
x_args = {
    #"reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    #"max_seq_length": 128,
    "num_train_epochs": 1,
    #"evaluate_during_training_steps": 1000,
    "wandb_project": "Sentiment-Analysis",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "use_early_stopping": True,
    "learning_rate": 1e-4,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

# Default VALUES of "args" parameters

self.args =

{
    
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

     "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": False,
    "reprocess_input_data": False,
    "evaluate_during_training": False,

    "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
}

In [21]:
model = ClassificationModel(model_type, model_name,num_labels = 2,args=model_args,use_cuda=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


This creates a ClassificationModel that can be used for training, evaluating, and predicting on Binary classification tasks. The first parameter is the model_type, the second is the model_name, and the third is the number of labels in the data.

model_type may be one of **['bert', 'xlnet', 'xlm', 'roberta', 'distilbert']**.


# TRAIN AND EVAL DATASET PREPARATION


In [22]:
train_df = pd.DataFrame({"review":x_train, "sentiment":y_train})
eval_df = pd.DataFrame({"review":x_test, "sentiment":y_test})

In [23]:
train_df.head()

,review,sentiment
368489,"These earrings are as pictured, very clear and...",1
407713,Fits perfectly and looks great,1
432274,"Looks like the real thing, & the salesperson a...",1
804096,Exactly what you would expect. Very cute.,1
502777,The necklace was not the chain in the picture....,1


In [24]:
!rm -rf outputs/

# TRAINING OF MODEL

In [ ]:
#model.train_model(train_df, eval_df=eval_df,acc=accuracy_score)
model.train_model(train_df,acc=accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/1256 [00:00<?, ?it/s]

# EVALUATION OF MODEL

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=accuracy_score)

The eval_model method is used to perform evaluation on an evaluation dataset. This method has three return values.

**result:** The evaluation result in the form of a dict.

**model_outputs:** A list of model outputs for each item in the evaluation dataset. This is useful if you need probabilities for each class rather than a single prediction. Note that a sigmoid function has been applied to each output to squash the values between 0 and 1.

**wrong_predictions:** A list of InputFeature of each incorrect prediction. The text may be obtained from the InputFeature.text_a attribute

In [ ]:
result

In [ ]:
x_test.shape

In [ ]:
model

In [ ]:
# Get the number of class labels
num_classes = len(model.config.label2id)

# Reshape x_test to have the correct number of columns
#x_test = x_test.to_numpy().reshape(-1, 1)

# PREDICTION OF MODEL

In [ ]:
preds, model_ouputs = model.predict(x_test)

In [ ]:
preds

In [ ]:
x_test.shape

In [ ]:
import numpy as np

input_data = model.predict([" i love u"])


In [ ]:
# import logging

# import pandas as pd
# import sklearn

# import wandb
# from simpletransformers.classification import (
#     ClassificationArgs,
#     ClassificationModel,
# )

# sweep_config = {
#     "method": "bayes",  # grid, random
#     "metric": {"name": "train_loss", "goal": "minimize"},
#     "parameters": {
#         "num_train_epochs": {"values": [2, 3, 5]},
#         "learning_rate": {"min": 5e-5, "max": 4e-4},
#     },
# }

# sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)


# model_args = ClassificationArgs()
# model_args.reprocess_input_data = True
# model_args.overwrite_output_dir = True
# model_args.evaluate_during_training = True
# model_args.manual_seed = 4
# model_args.use_multiprocessing = False
# model_args.train_batch_size = 16
# model_args.eval_batch_size = 8
# model_args.labels_list = ["1", "0"]
# model_args.wandb_project = "Simple Sweep"

# def train():
#     # Initialize a new wandb run
#     wandb.init()

#     # Create a TransformerModel
#     model = ClassificationModel(
#         "bert", "bert-base-cased",
#         use_cuda=True,
#         args=model_args,
#         sweep_config=wandb.config,
#     )

#     # Train the model
#     model.train_model(train_df, eval_df=eval_df)

#     # Evaluate the model
#     model.eval_model(eval_df)

#     # Sync wandb
#     wandb.join()


# wandb.agent(sweep_id, train)